## Voronoi expansion implementation in DEVS Simulation Framework 


In [1]:
%load_ext autoreload
%autoreload 2

### Attemping to make an animated matplotlib celldevs plot
Using as example project a Vornoi Zones implementation in cellDevs.

See this links.
- http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/
- https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.load.html
- https://www.numpy.org/devdocs/reference/generated/numpy.lib.format.html
- https://github.com/rogersce/cnpy

In [2]:
import numpy as np
import matplotlib

from matplotlib import pyplot as plt

from pycdevs import cdppwrapper as cdppExceptions
from pycdevs import voronoi

from pycdevs.cdppwrapper import CDPPWrapper, DrawlogFailedException
from pycdevs.models import Model, CellValue
from pycdevs.jupyter import get_simulated_voronoi_animation

matplotlib.rcParams['figure.figsize'] = (10,8)

Snippet to generate a randomly initiated cell values, for a voronoi model of a certain size

In [3]:
import random

inital_cell_values = np.zeros((40, 40), dtype=int)
get_random_i = lambda : random.randint(0, inital_cell_values.shape[0]-1)
get_random_j = lambda : random.randint(0, inital_cell_values.shape[1]-1)
current_value = 1

for _ in range(5):
    random_i = get_random_i()
    random_j = get_random_j()
    inital_cell_values[random_i, random_j] = current_value
    print("Initiating ({0},{1}) with value {2}".format(random_i, random_j, current_value))
    current_value += 1
    
devs_initiall_cell_values = CellValues(inital_cell_values, 0)

Initiating (39,0) with value 1
Initiating (29,34) with value 2
Initiating (24,9) with value 3
Initiating (14,17) with value 4
Initiating (18,21) with value 5


In [4]:
# diamond shaped neighbourhood
diamond_neighbours = [
    (0, -2), \
    (-1,-1), (0, -1), (1,-1), \
    (-2, 0), (-1,0), (1,0), (2, 0),\
    (-1,1), (0, 1), (1,1), \
    (0, 2)
]

diamond_neighbourhood = voronoi.Neighbourhood(diamond_neighbours)

simple_voronoi_rule = "rule : {${current_cell}} 100 { (0,0)=0 and ${current_cell}!=0 and ${current_cell}!=? }"
simple_voronoi_rule_builder = voronoi.RulesetBuilder(simple_voronoi_rule)

diamond_voronoi_model = voronoi.VoronoiModel("diamondVoronoi", diamond_neighbourhood,\
                                             devs_initiall_cell_values, simple_voronoi_rule_builder).build()

In [5]:
wrapper = CDPPWrapper(diamond_voronoi_model, '00:01:00:00')
# for debug reasons, print CD++ executed command
print("" + " ".join(wrapper.getArguments()))

try:
    wrapper.run()
    drawlogNPFilename = wrapper.drawlog('00:00:00:100')    

# Simulator failure
except cdppExceptions.SimulationExecutedButFailedException:
    print(wrapper.getSimulationOutput())

# Drawlog parsing failure
except DrawlogFailedException as e:
    print(e.stderr.decode('ascii'))

cd++ -m/tmp/1543792922812.ma -o/tmp/1543792922812.out -t00:01:00:00 -l/tmp/1543792922812.log


Have to extract all this drawing logic in some kind of module, of Juputer extension. Its a lot of boilerplate.

In summary, what it does is:

1. Load drawlog-generated numpy files, with the drawn simulation
2. Conglomerate all loaded matrixes in a single one

In [17]:
%%capture
animation = get_simulated_voronoi_animation(drawlogNPFilename, devs_initiall_cell_values)

In [18]:
animation

Iterate over several neighbourhoods, and generate animation for each, to compare them side by side.

Maybe, a neighbourhood object can be defined, and passed as parameter to the model.